In [ ]:
import rasterio
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
import hdbscan

In [ ]:
with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/cropped.tif') as src:
    data = src.read()
    image = src.read(1)
    cols, rows = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    x = np.array(xs)
    x = (x-np.min(x))/(np.max(x)-np.min(x))
    y = np.array(ys)
    y = (y-np.min(y))/(np.max(y)-np.min(y))
    profile = src.profile

pixels = image.reshape(-1, 1)
pixels = np.clip(pixels, 0.5, pixels.max())
pixels = pixels/np.max(pixels)
print(pixels)

In [ ]:
data_points = []

for i in range(image.shape[0]):
    # Iterate through each column in the current row
    for j in range(image.shape[1]):
        data_points.append([x[i, j]*2, y[i, j]*2, image[i, j]])

data_points

In [ ]:
# with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/cropped.tif') as src:
#     image = src.read()

# pixels = image.reshape(image.shape[0], -1).T


In [ ]:
print(image.shape)

In [ ]:
# clstr = KMeans(50).fit(data_points)
# clstr = DBSCAN(eps=3,min_samples=500).fit(data_points)

# from multiprocessing import set_start_method, get_context
# set_start_method('spawn')

clusterer = hdbscan.HDBSCAN(min_cluster_size=50)
clstr= clusterer.fit_predict(data_points)

In [ ]:
np.shape(clstr)
#removed.labels_ for hdbscan

In [ ]:
unique_vals, counts = np.unique(clstr, return_counts=True)
#removed.labels_ for hdbscan
print(dict(zip(unique_vals, counts)))

In [ ]:
labels = clstr.reshape(image.shape[0], image.shape[1])
#removed.labels_ for hdbscan
np.shape(labels)

In [ ]:
with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/clustered.tif', 'w+', driver='GTiff', width=image.shape[1], height=image.shape[0], count=2, dtype=labels.dtype, crs=src.crs, transform=src.transform) as dst:
    dst.write(image,1)
    # Write the cluster labels to a new band
    dst.write(labels,2)

# with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/clustered3.tif', 'w', driver='GTiff', width=image.shape[2], height=image.shape[1], count=1, dtype=np.uint8) as dst:
#     dst.write(labels_2d.astype(np.uint8), 1)
